In [1]:
# 导入必要的库
import pandas as pd  # 用于数据处理和分析
from datetime import datetime  # 用于处理日期时间
import matplotlib.pyplot as plt  # 用于数据可视化
from datetime import datetime  # datetime重复导入，可以删除一个


In [7]:
# 读取用户行为数据集，将所有列都以字符串类型读取
data_user = pd.read_csv('tianchi_mobile_recommend_train_user.csv',dtype=str)
print(data_user.shape)  # 打印数据集的形状（行数和列数）
data_user.info()  # 显示数据集的基本信息，包括列名、数据类型和内存使用情况

(12256906, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   item_id        object
 2   behavior_type  object
 3   user_geohash   object
 4   item_category  object
 5   time           object
dtypes: object(6)
memory usage: 561.1+ MB


In [8]:
# 数据预处理：时间特征提取和转换
# 从time列中分离出日期和小时
data_user['date'] = data_user['time'].str[0:10]  # 提取日期部分（前10个字符）
data_user['hour'] = data_user['time'].str[11:]   # 提取小时部分（第11个字符之后）

# 将字符串类型的时间数据转换为合适的数据类型
data_user['date'] = pd.to_datetime(data_user['date'])  # 转换为datetime类型
data_user['time']= pd.to_datetime(data_user['time'])   # 转换为datetime类型
data_user['hour'] = data_user['hour'].astype(int)      # 转换为整数类型

In [10]:
data_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   item_id        object        
 2   behavior_type  object        
 3   user_geohash   object        
 4   item_category  object        
 5   time           datetime64[ns]
 6   date           datetime64[ns]
 7   hour           int64         
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 748.1+ MB


In [11]:
# 计算每个用户的最近购买时间（Recency）
# behavior_type为4表示购买行为
# 对每个用户，找出其最后一次购买的日期，然后用2014-12-18减去这个日期，得到距离最近购买的天数
recent_buy_time=data_user[data_user['behavior_type'] == '4'].groupby ('user_id')['date'].apply(lambda x:datetime(2014,12,18)- x.sort_values().iloc[-1])

In [12]:
recent_buy_time

user_id
100001878    0 days
100011562    2 days
100012968    0 days
100014060    0 days
100024529    2 days
              ...  
99960313     5 days
9996155     11 days
99963140     3 days
99968428     4 days
99989881     3 days
Name: date, Length: 8886, dtype: timedelta64[ns]

In [13]:
# 处理最近购买时间数据
recent_buy_time=recent_buy_time.reset_index().rename(columns={'date':'recent'})  # 重置索引并重命名列
recent_buy_time['recent']=recent_buy_time.recent.apply(lambda x: x.days)  # 将时间间隔转换为天数
recent_buy_time.describe()  # 查看最近购买时间的统计描述，包括均值、标准差、分位数等


,recent
count,8886.000000
mean,5.811839
std,6.678478
min,0.000000
25%,1.000000
50%,4.000000
75%,7.000000
max,30.000000


In [15]:
# 计算购买频率（Frequency）
# 使用nunique()统计每个用户在不同日期的购买次数
# count和nunique的区别：count统计总购买次数，nunique统计不同日期的购买次数
buy_freq = data_user[data_user ['behavior_type'] =='4'].groupby('user_id').date.nunique()

buy_freq  # 显示每个用户的购买频率

user_id
100001878    15
100011562     3
100012968    11
100014060    12
100024529     9
             ..
99960313      5
9996155       3
99963140      8
99968428      9
99989881     13
Name: date, Length: 8886, dtype: int64

In [17]:
# RFM分析：将用户按照最近购买时间和购买频率进行分类
buy_freq=buy_freq.reset_index().rename(columns={'date':'freq'})  # 重置索引并重命名列
rfm= pd.merge (recent_buy_time, buy_freq,left_on ='user_id' ,right_on ='user_id')  # 合并最近购买时间和购买频率数据

# 使用分位数将用户分为两类
rfm['recent_value'] = pd.qcut(rfm.recent,2,labels=['2','1'])  # 最近购买时间分类：1表示较旧，2表示较新
rfm['freq_value'] = pd.qcut(rfm.freq,2,labels=['1','2'])     # 购买频率分类：1表示较低，2表示较高

# 组合R和F值形成最终的客户分类
rfm['rfm'] = rfm['recent_value'].str.cat(rfm['freq_value'])  # 将R值和F值连接成两位数字
rfm  # 显示完整的RFM分析结果


,user_id,recent,index,freq,recent_value,freq_value,rfm
0,100001878,0,0,15,2,2,22
1,100011562,2,1,3,2,1,21
2,100012968,0,2,11,2,2,22
3,100014060,0,3,12,2,2,22
4,100024529,2,4,9,2,2,22
...,...,...,...,...,...,...,...
8881,99960313,5,8881,5,1,1,11
8882,9996155,11,8882,3,1,1,11
8883,99963140,3,8883,8,2,2,22
8884,99968428,4,8884,9,2,2,22
